In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
model_name = "EleutherAI/gpt-j-6B"
use_gpu = True
num_workers = 2
cpus_per_worker = 4

In [3]:
import ray

ray.init(
    runtime_env={
        "pip": [
            "datasets",
            "evaluate",
            # Latest combination of accelerate==0.19.0 and transformers==4.29.0
            # seems to have issues with DeepSpeed process group initialization,
            # and will result in a batch_size validation problem.
            # TODO(jungong) : get rid of the pins once the issue is fixed.
            "accelerate==0.16.0",
            "transformers==4.26.0",
            "torch>=1.12.0",
            "deepspeed==0.12.3",
        ],
    },
)

2024-01-24 09:33:47,352	WARNING utils.py:575 -- Detecting docker specified CPUs. In previous versions of Ray, CPU detection in containers was incorrect. Please ensure that Ray has enough CPUs allocated. As a temporary workaround to revert to the prior behavior, set `RAY_USE_MULTIPROCESSING_CPU_COUNT=1` as an env var before starting Ray. Set the env var: `RAY_DISABLE_DOCKER_CPU_WARNING=1` to mute this warning.
2024-01-24 09:33:47,357	WARNING utils.py:587 -- Ray currently does not support initializing Ray with fractional cpus. Your num_cpus will be truncated from 40.8 to 40.
2024-01-24 09:33:48,733	INFO worker.py:1724 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.9.1


In [4]:
from datasets import load_dataset

print("Loading tiny_shakespeare dataset")
current_dataset = load_dataset("tiny_shakespeare")
current_dataset

Loading tiny_shakespeare dataset


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1
    })
})

In [5]:
import ray.data

ray_datasets = {
    "train": ray.data.from_huggingface(current_dataset["train"]),
    "validation": ray.data.from_huggingface(current_dataset["validation"])
}

ray_datasets

{'train': MaterializedDataset(num_blocks=1, num_rows=1, schema={text: string}),
 'validation': MaterializedDataset(num_blocks=1, num_rows=1, schema={text: string})}

In [6]:
block_size = 512

In [7]:
from transformers import AutoTokenizer

def split_text(batch: pd.DataFrame) -> pd.DataFrame:
    text = list(batch["text"])
    flat_text = "".join(text)
    split_text = [
        x.strip()
        for x in flat_text.split("\n")
        if x.strip() and not x.strip()[-1] == ":"
    ]
    return pd.DataFrame(split_text, columns=["text"])


def tokenize(batch: pd.DataFrame) -> dict:
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    tokenizer.pad_token = tokenizer.eos_token
    ret = tokenizer(
        list(batch["text"]),
        truncation=True,
        max_length=block_size,
        padding="max_length",
        return_tensors="np",
    )
    ret["labels"] = ret["input_ids"].copy()
    return dict(ret)

processed_datasets = {
    key: ds.map_batches(split_text, batch_format="pandas").map_batches(tokenize, batch_format="pandas").random_shuffle(seed=42)
    for key, ds in ray_datasets.items()
}
processed_datasets

{'train': RandomShuffle
 +- MapBatches(tokenize)
    +- MapBatches(split_text)
       +- Dataset(num_blocks=1, num_rows=1, schema={text: string}),
 'validation': RandomShuffle
 +- MapBatches(tokenize)
    +- MapBatches(split_text)
       +- Dataset(num_blocks=1, num_rows=1, schema={text: string})}

In [8]:
import evaluate
import torch
from transformers import (
    Trainer,
    TrainingArguments,
    GPTJForCausalLM,
    AutoTokenizer,
    default_data_collator,
)
from transformers.utils.logging import disable_progress_bar, enable_progress_bar

from ray import train
from ray.train.huggingface.transformers import (
    prepare_trainer,
    RayTrainReportCallback
)


def train_func(config):
    # Use the actual number of CPUs assigned by Ray
    os.environ["OMP_NUM_THREADS"] = str(
        train.get_context().get_trial_resources().bundles[-1].get("CPU", 1)
    )
    # Enable tf32 for better performance
    torch.backends.cuda.matmul.allow_tf32 = True

    batch_size = config.get("batch_size", 4)
    epochs = config.get("epochs", 2)
    warmup_steps = config.get("warmup_steps", 0)
    learning_rate = config.get("learning_rate", 0.00002)
    weight_decay = config.get("weight_decay", 0.01)
    steps_per_epoch = config.get("steps_per_epoch")

    deepspeed = {
        "fp16": {
            "enabled": "auto",
            "initial_scale_power": 8,
        },
        "bf16": {"enabled": "auto"},
        "optimizer": {
            "type": "AdamW",
            "params": {
                "lr": "auto",
                "betas": "auto",
                "eps": "auto",
            },
        },
        "zero_optimization": {
            "stage": 3,
            "offload_optimizer": {
                "device": "cpu",
                "pin_memory": True,
            },
            "offload_param": {
                "device": "cpu",
                "pin_memory": True,
            },
            "overlap_comm": True,
            "contiguous_gradients": True,
            "reduce_bucket_size": "auto",
            "stage3_prefetch_bucket_size": "auto",
            "stage3_param_persistence_threshold": "auto",
            "gather_16bit_weights_on_model_save": True,
            "round_robin_gradients": True,
        },
        "gradient_accumulation_steps": "auto",
        "gradient_clipping": "auto",
        "steps_per_print": 10,
        "train_batch_size": "auto",
        "train_micro_batch_size_per_gpu": "auto",
        "wall_clock_breakdown": False,
    }

    print("Preparing training arguments")
    training_args = TrainingArguments(
        "output",
        logging_steps=1,
        save_strategy="steps",
        save_steps=steps_per_epoch,
        max_steps=steps_per_epoch * epochs,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        warmup_steps=warmup_steps,
        label_names=["input_ids", "attention_mask"],
        push_to_hub=False,
        report_to="none",
        disable_tqdm=True,  # declutter the output a little
        fp16=True,
        gradient_checkpointing=True,
        deepspeed=deepspeed,
    )
    disable_progress_bar()

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    print("Loading model")

    model = GPTJForCausalLM.from_pretrained(model_name, use_cache=False)
    model.resize_token_embeddings(len(tokenizer))

    print("Model loaded")

    enable_progress_bar()

    metric = evaluate.load("accuracy")

    train_ds = train.get_dataset_shard("train")
    eval_ds = train.get_dataset_shard("validation")

    train_ds_iterable = train_ds.iter_torch_batches(batch_size=batch_size)
    eval_ds_iterable = eval_ds.iter_torch_batches(batch_size=batch_size)

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds_iterable,
        eval_dataset=eval_ds_iterable,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
    )

    # Add callback to report checkpoints to Ray Train
    trainer.add_callback(RayTrainReportCallback())
    trainer = prepare_trainer(trainer)
    trainer.train()

In [9]:
from ray.train.torch import TorchTrainer
from ray.train import RunConfig, ScalingConfig

batch_size = 16
train_ds_size = processed_datasets["train"].count()
steps_per_epoch = train_ds_size // (batch_size * num_workers)

trainer = TorchTrainer(
    train_loop_per_worker=train_func,
    train_loop_config={
        "epochs": 1,
        "batch_size": batch_size,  # per device
        "steps_per_epoch": steps_per_epoch
    },
    scaling_config=ScalingConfig(
        num_workers=num_workers,
        use_gpu=use_gpu,
        resources_per_worker={"GPU": 1, "CPU": cpus_per_worker},
    ),
    datasets=processed_datasets,
    run_config=RunConfig(storage_path="/models"),
)

2024-01-24 09:34:11,896	INFO streaming_executor.py:112 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[MapBatches(split_text)->MapBatches(tokenize)->RandomShuffle]
2024-01-24 09:34:11,898	INFO streaming_executor.py:113 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), exclude_resources=ExecutionResources(cpu=0, gpu=0, object_store_memory=0), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-01-24 09:34:11,901	INFO streaming_executor.py:115 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


- MapBatches(split_text)->MapBatches(tokenize)->RandomShuffle 1:   0%|          | 0/1 [00:00<?, ?it/s]

Shuffle Map 2:   0%|          | 0/1 [00:00<?, ?it/s]

Shuffle Reduce 3:   0%|          | 0/1 [00:00<?, ?it/s]

Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
results = trainer.fit()

(TrainTrainable pid=266117) [2024-01-24 09:34:31,353] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


(RayTrainWorker pid=266237) Setting up process group for: env:// [rank=0, world_size=2]
(TorchTrainer pid=266117) Started distributed worker processes: 
(TorchTrainer pid=266117) - (ip=192.168.64.2, pid=266237) world_rank=0, local_rank=0, node_rank=0
(TorchTrainer pid=266117) - (ip=192.168.64.2, pid=266238) world_rank=1, local_rank=1, node_rank=0


(RayTrainWorker pid=266237) [2024-01-24 09:34:40,446] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
(RayTrainWorker pid=266238) [2024-01-24 09:34:40,440] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


(SplitCoordinator pid=266453) Auto configuring locality_with_output=['f04480c4dea75141f1a311fae660b2c4899ab20f4cce2c91bfa4affb', 'f04480c4dea75141f1a311fae660b2c4899ab20f4cce2c91bfa4affb']


(RayTrainWorker pid=266237) Preparing training arguments
(RayTrainWorker pid=266237) [2024-01-24 09:34:41,633] [INFO] [comm.py:637:init_distributed] cdb=None
(RayTrainWorker pid=266237) Loading model
(RayTrainWorker pid=266237) [2024-01-24 09:35:04,380] [INFO] [partition_parameters.py:348:__exit__] finished initializing model - num_params = 285, num_elems = 6.05B
(RayTrainWorker pid=266238) Preparing training arguments
(RayTrainWorker pid=266238) [2024-01-24 09:34:41,632] [INFO] [comm.py:637:init_distributed] cdb=None
(RayTrainWorker pid=266238) Loading model
(RayTrainWorker pid=266237) Model loaded
(RayTrainWorker pid=266237) [2024-01-24 09:35:27,260] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.12.3, git-hash=unknown, git-branch=unknown
(RayTrainWorker pid=266237) [2024-01-24 09:35:27,280] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False


...

(RayTrainWorker pid=266238) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/models/TorchTrainer_2024-01-24_09-34-24/TorchTrainer_c3062_00000_0_2024-01-24_09-34-25/checkpoint_000000)
(RayTrainWorker pid=266238) /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1879: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
(RayTrainWorker pid=266238)   warnings.warn(
(RayTrainWorker pid=266237) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/models/TorchTrainer_2024-01-24_09-34-24/TorchTrainer_c3062_00000_0_2024-01-24_09-34-25/checkpoint_000000)


(RayTrainWorker pid=266237) {'train_runtime': 8178.4812, 'train_samples_per_second': 1.319, 'train_steps_per_second': 0.082, 'train_loss': 0.10651105660005564, 'epoch': 1.0}


(RayTrainWorker pid=266237) 
(RayTrainWorker pid=266237) 
(RayTrainWorker pid=266237) Training completed. Do not forget to share your model on huggingface.co/models =)
(RayTrainWorker pid=266237) 
(RayTrainWorker pid=266237) 


(RayTrainWorker pid=266237) Adam Optimizer #0 is created with AVX512 arithmetic capability.
(RayTrainWorker pid=266237) Config: alpha=0.000020, betas=(0.900000, 0.999000), weight_decay=0.000000, adam_w=1


2024-01-24 11:56:20,364	WARNING experiment_state.py:307 -- Syncing the experiment checkpoint to cloud took a long time with 221.05 seconds. This can be due to a large number of trials, large logfiles, or throttling from the remote storage provider for too frequent syncs. If your `CheckpointConfig.num_to_keep` is a low number, this can trigger frequent syncing, in which case you should increase it. 
2024-01-24 11:56:20,563	INFO tune.py:1042 -- Total run time: 8515.61 seconds (8294.29 seconds for the tuning loop).


In [11]:
checkpoint = results.checkpoint
checkpoint

Checkpoint(filesystem=local, path=/models/TorchTrainer_2024-01-24_09-34-24/TorchTrainer_c3062_00000_0_2024-01-24_09-34-25/checkpoint_000000)

In [13]:
checkpoint.to_directory('new_models')

'new_models'

In [14]:
from transformers import pipeline, AutoTokenizer, GPTJForCausalLM

model = GPTJForCausalLM.from_pretrained("new_models/checkpoint")
tokenizer = AutoTokenizer.from_pretrained("new_models/checkpoint")
pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [15]:
for sentence in pipe(["Romeo and Juliet", "Romeo", "Juliet"], do_sample=True, min_length=20):
    print(sentence)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Romeo and Juliet doff their names for mine,--or, if you can, my office;'}]
[{'generated_text': 'Romeo is slain. What of that? he did but owe thy life, my wife,'}]
[{'generated_text': 'Julietta, my child is dead to-night, she said; that she was dead.'}]
